# Butterfly Cnn

In [ ]:
#import libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.callbacks import EarlyStopping

from PIL import Image
import pandas as pd
import requests
import numpy as np

## importing data

In [ ]:
#load in datasets
train_df = pd.read_csv('Resources/Training_set.csv')
test_df = pd.read_csv('Resources/Testing_set.csv')
train_path = 'Resources/train/'
test_path = 'Resources/test/'

In [ ]:
#The data was split into train and test sets and I would like to have it just be one dataset
butterfly_df = pd.concat([train_df, test_df], axis=0).reset_index(drop=True)

In [ ]:
butterfly_df.head()

In [ ]:
#Before moving forward I need to also combine the train and test folder for the images
#But I also need to make sure each image is correctly placed with the right label
# Initialize list to store images
images = []

# Loop through each entry in the combined DataFrame
for i in range(len(butterfly_df)):
    filename = butterfly_df.iloc[i, 0]
    
    if i < len(train_df):
        path = train_path + filename  # First part of butterfly_df is from train_df
    else:
        path = test_path + filename   # Second part of butterfly_df is from test_df

    print(f'{i+1} of {len(butterfly_df)}: Attempting to import {filename}')
    
    try:
        # Open, add image to list, and close to free up file handle
        with Image.open(path) as img:
            images.append(img.copy())
    except Exception as e:
        print(f'FAILED to load {filename}: {e}')

## Preprocessing

In [ ]:
# Print a random image from the list to ensure the import was successful
images[6543]

In [ ]:
# Check the size of the second image
images[1].size

In [ ]:
#if I need to make it smaller if my machine is not up to par for doing it with 224 by 224.
#it will be more accurate at that size though. Also can do 128 by 128

# Get all the sizes into a list, then convert to a set
sizes = set([img.size for img in images])
sizes
# Use a for loop to resize all images to 64 by 60
target_size = (128, 128)

resized_images = [img.resize(target_size, resample = Image.LANCZOS) for img in images]
resized_images[1]

In [ ]:
# Convert all images to floating point numpy arrays
float_images = [np.array(img).astype(np.float32) for img in resized_images]

# Display the pixel values of the first image
print("Pixel Values:")
print(float_images[0])

In [ ]:
# To normalize images to a range between 0 and 1,
# we need to divide all pixel values by the max of 255

normalized_images = [img/255 for img in float_images]

# Display the pixel values of the first image
print("Pixel Values:")
print(normalized_images[0])

## Labels

In [ ]:
# Print the first few image filenames
butterfly_df.head()

In [ ]:
# First, remove the .jpg file extension, then split into new columns. 
# Remove the .jpg extension
butterfly_df['filename'] = butterfly_df['filename'].str.replace('.jpg', '', regex=False)
butterfly_df.head()

In [ ]:
# Now we can call our preprocessed pixel data 'X'
X = normalized_images

# For our purposes, we'll select the userid column as 'y'
y = butterfly_df['label']

In [ ]:
# Check the total number of classes
y.nunique()

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
# Now we'll split our data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

## Augmentation

In [ ]:
# Apply augmentation to the whole training dataset
# Define the augmentation pipeline
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.2),         # Randomly rotate images by 20%
    tf.keras.layers.RandomTranslation(0.2, 0.2), # Randomly shift images horizontally and vertically by 20%
    tf.keras.layers.RandomZoom(0.3),             # Randomly zoom images by up to 30%
    tf.keras.layers.RandomFlip('horizontal'),    # Random horizontal flip
    tf.keras.layers.RandomContrast(0.2)          # Random contrast adjustment
])

# Create variables to hold the X and y training data
X_train_aug = []
y_train_aug = []

# Loop through all the images.
for i in range(len(X_train)):
    # Select the image
    img = X_train[i]
    # Select the label from the training data
    label = y_train[i]

    # Ensure that the input data has the correct shape
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    # Add 5 images for every original image
    for j in range(5):
        # Append a new image to the X list
        X_train_aug.append(data_augmentation(img, training=True)[0].numpy())
        # Append the label for the original image to the y list
        y_train_aug.append(label)

# Print the length of each list
print(len(X_train_aug))
print(len(y_train_aug))

In [ ]:
# Reshape test data for the model
X_test_np = []
for img in X_test:
    # Append the image to the list
    X_test_np.append(img)

# Convert to numpy array
X_test_np = np.array(X_test_np)

# Check the shape of the first image
X_test_np[0].shape

## Creating the Model

In [ ]:
# One hot encode the y data
y_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False).fit(np.array(y_train_aug).reshape(-1, 1))
y_train_aug_enc = y_encoder.transform(np.array(y_train_aug).reshape(-1, 1))
y_test_enc = y_encoder.transform(np.array(y_test).reshape(-1, 1))

# Convert values to numpy arrays
X_train_aug_np = np.array(X_train_aug)
X_test_np = np.array(X_test_np)
y_train_aug_np = np.array(y_train_aug_enc)
y_test_np = np.array(y_test_enc)

# Load and preprocess your CMU Face Images dataset (Ensure each image is labeled as "with sunglasses" or "without sunglasses")
# The following code assumes that you have already loaded and preprocessed your dataset into 'X' and 'y' (features and labels).

# Split the training dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_aug_np, y_train_aug_np, test_size=0.2, random_state=42)

# Print the total number of one_hot_encoded columns
np.array(y_train).shape

In [ ]:
# Define a CNN model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),  # Dropout layer with 20% dropout rate

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),  # Dropout layer with 30% dropout rate

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.4),  # Dropout layer with 40% dropout rate

    layers.Dense(76, activation='sigmoid')  # Output layer for 76 classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping with patience of 3 epochs
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=[early_stopping]
)

In [ ]:
#evaluate the model
model.evaluate(X_test_np, y_test_np)

In [ ]:
# model = load_model("butterfly_model.h5")